# 1.1.1.2: Operaciones Vectoriales, Normas y Span

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Calcular y visualizar** la suma, resta y multiplicación por un escalar de vectores.
- **Explicar** el concepto de **vectorización** y por qué es crucial para el rendimiento computacional.
- **Calcular, diferenciar e interpretar** las normas **L1 (Manhattan)** y **L2 (Euclidiana)** de un vector.
- **Conectar** las normas vectoriales con las métricas de error (MAE, RMSE) y la regularización (Lasso, Ridge) en Machine Learning.
- **Comprender, calcular y visualizar** los conceptos de **Combinación Lineal** y **Espacio Generado (Span)**.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

def plot_vectors_2d(vectors, origins=None, colors=None, legends=None, title="Gráfico de Vectores 2D", figsize=(7,7)):
    """Función mejorada para graficar vectores 2D, permitiendo orígenes personalizados."""
    fig, ax = plt.subplots(figsize=figsize)
    if colors is None:
        colors = ['#0072B2', '#E69F00', '#D55E00', '#CC79A7', '#56B4E9', '#009E73']
    if origins is None:
        origins = [[0, 0]] * len(vectors)
    elif not isinstance(origins[0], list) and not isinstance(origins[0], np.ndarray):
        origins = [origins] * len(vectors)
    
    for i, v in enumerate(vectors):
        origin = origins[i]
        ax.quiver(origin[0], origin[1], v[0], v[1], angles='xy', scale_units='xy', scale=1, 
                  color=colors[i % len(colors)], label=legends[i] if legends and i < len(legends) else str(v))

    all_points = np.array(origins + [np.array(o) + np.array(v) for o, v in zip(origins, vectors)])
    x_min, y_min = all_points.min(axis=0)
    x_max, y_max = all_points.max(axis=0)
    buffer = max(abs(x_min), abs(x_max), abs(y_min), abs(y_max)) * 0.1 + 1
    ax.set_xlim(x_min - buffer, x_max + buffer)
    ax.set_ylim(y_min - buffer, y_max + buffer)
    ax.set_aspect('equal'); ax.grid(True); ax.legend()
    ax.set_title(title)
    plt.show()

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

Al igual que en el notebook anterior, usaremos nuestro arsenal de funciones para generar datos. Esto nos da la flexibilidad de crear contextos ricos y variados para cada concepto matemático.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_geometric_shapes import create_geometric_shapes
from src.data_generation.create_business_data import create_business_data
from src.data_generation.create_edge_cases import create_edge_cases

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor
# Continuamos usando el rendimiento estudiantil para contextualizar operaciones. 
# Por ejemplo, sumar vectores de 'mejora' o promediar perfiles de estudiantes.
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=100)

# 💡 CONTEXTO PEDAGÓGICO: Aplicación a Negocios
# Ideal para entender normas. Un producto con una norma L2 grande es un 'extremo' en el mercado. 
# La resta de vectores puede representar la 'diferencia de perfil' entre dos productos.
datos_negocio = create_business_data(rng, simplified=True, n_samples=75)

# 💡 CONTEXTO PEDAGÓGICO: Intuición Geométrica
# Usaremos vectores de formas geométricas para visualizar conceptos como Span y combinaciones lineales
# de una forma pura, sin el 'ruido' de un contexto del mundo real.
vec_base_1 = vector([2, 1])
vec_base_2 = vector([-1, 2]) # Casi ortogonales para un buen span

print("Datasets generados y listos para usar.")
datos_estudiantes.head()

---
## 1. Operaciones Vectoriales Fundamentales

### Suma y Resta

Estas operaciones se realizan **componente a componente** y solo son posibles entre vectores de la **misma dimensión**.

- **Suma**: $\vec{v} + \vec{w} = [v_1+w_1, v_2+w_2, \dots, v_n+w_n]$
- **Resta**: $\vec{v} - \vec{w} = [v_1-w_1, v_2-w_2, \dots, v_n-w_n]$

Geométricamente, la suma $\vec{v} + \vec{w}$ sigue la **regla del paralelogramo**. La resta $\vec{v} - \vec{w}$ es el vector que va desde la punta de $\vec{w}$ hasta la punta de $\vec{v}$.

### Multiplicación por un Escalar

Multiplicar un vector $\vec{v}$ por un número (escalar) $c$ **escala** (estira o encoge) su magnitud. Si $c < 0$, también invierte su dirección.

$ c\vec{v} = [c \cdot v_1, c \cdot v_2, ..., c \cdot v_n] $

### Ejemplo Demostrativo 1: Visualización de Operaciones

In [ ]:
v = np.array([3, 1])
w = np.array([1, 2])

# Operaciones
v_plus_w = v + w
v_minus_w = v - w
scaled_v = 2 * v

# --- Visualización de la Suma (Regla del Paralelogramo) ---
fig, ax = plt.subplots(1, 2, figsize=(14, 7))

# Orígenes y vectores para el paralelogramo
origins = [[0,0], [0,0], [0,0], v, w] # Para dibujar el paralelogramo
vectors = [v, w, v_plus_w, w, v]
legends = [f'v={v}', f'w={w}', f'v+w={v_plus_w}', '', '']
colors = ['#0072B2', '#E69F00', '#D55E00', '#0072B2', '#E69F00']

for i, vec in enumerate(vectors):
    ax[0].quiver(origins[i][0], origins[i][1], vec[0], vec[1], angles='xy', scale_units='xy', scale=1, 
                 color=colors[i], linestyle='--' if i > 2 else '-')
ax[0].text(v_plus_w[0], v_plus_w[1], 'v+w', fontsize=12)
ax[0].set_title('Suma de Vectores (Regla del Paralelogramo)')

# --- Visualización de Resta y Escalamiento ---
ax[1].quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1, color='#0072B2', label=f'v={v}')
ax[1].quiver(0, 0, scaled_v[0], scaled_v[1], angles='xy', scale_units='xy', scale=1, color='#56B4E9', label=f'2v={scaled_v}')
ax[1].quiver(w[0], w[1], v_minus_w[0], v_minus_w[1], angles='xy', scale_units='xy', scale=1, color='#CC79A7', label=f'v-w={v_minus_w}')
ax[1].quiver(0,0, v_minus_w[0], v_minus_w[1], angles='xy', scale_units='xy', scale=1, color='#CC79A7', linestyle=':')
ax[1].set_title('Resta y Multiplicación por Escalar')

for axis in ax:
    axis.set_xlim(0, 7); axis.set_ylim(0, 5)
    axis.set_aspect('equal'); axis.grid(True); axis.legend()

plt.show()

### 🚀 El Eje Computacional: La Magia de la Vectorización

Podríamos sumar vectores usando un bucle `for` de Python, pero en la práctica **nunca** lo hacemos. Las librerías como NumPy son órdenes de magnitud más rápidas gracias a la **vectorización**: NumPy delega la operación a rutinas de bajo nivel pre-compiladas y altamente optimizadas (escritas en C o Fortran), que operan en bloques enteros de memoria a la vez.

In [ ]:
# Creamos dos vectores grandes con 1 millón de elementos
vec_a_np = rng.random(1_000_000)
vec_b_np = rng.random(1_000_000)

# Método 1: Suma vectorizada de NumPy (rápido)
print("Midiendo el tiempo de la suma vectorizada de NumPy:")
%timeit vec_c_np = vec_a_np + vec_b_np

# Método 2: Suma con un bucle for de Python (lento)
print("\nMidiendo el tiempo de la suma con un bucle de Python:")
%timeit vec_c_loop = np.array([a + b for a, b in zip(vec_a_np, vec_b_np)])

print("\nObserva la diferencia de velocidad. ¡Ese es el poder de la vectorización!")

---
## 2. Medición de Vectores: Normas L1 y L2

Una **norma** es una función que asigna una "longitud" o "tamaño" estrictamente positivo a cada vector. Las más comunes en ciencia de datos son L1 y L2.

#### Norma L2 (Euclidiana) 
Es la distancia en línea recta desde el origen. Se calcula con el Teorema de Pitágoras.
$$ \|\vec{v}\|_2 = \sqrt{\sum_{i=1}^{n} v_i^2} $$

#### Norma L1 (Manhattan) 
Es la distancia recorriendo los ejes, como un taxi en una ciudad con calles en cuadrícula. Es la suma de los valores absolutos de los componentes.
$$ \|\vec{v}\|_1 = \sum_{i=1}^{n} |v_i| $$

> **Aplicaciones en Machine Learning:**
> - **Métricas de Error:** El **Error Absoluto Medio (MAE)** usa la norma L1 de los errores, haciéndolo robusto a outliers. El **Error Cuadrático Medio (RMSE)** usa la norma L2, que penaliza más fuertemente los errores grandes.
> - **Regularización:** La **Regularización Lasso (L1)** puede forzar a que los coeficientes de un modelo se vuelvan **exactamente cero** (selección de features). La **Regularización Ridge (L2)** encoge los coeficientes, pero raramente a cero.

### Ejemplo Demostrativo 2: Normas en un Contexto de Negocios

In [ ]:
# 1. GENERACIÓN DE DATOS (Ya generados)
# Usamos 'datos_negocio'

# 2. APLICACIÓN DEL CONCEPTO
# Buscamos productos 'extremos': el de mayor precio y el de mayor venta.
prod_max_precio = datos_negocio.loc[datos_negocio['precio'].idxmax()]
prod_max_ventas = datos_negocio.loc[datos_negocio['ventas_mensuales'].idxmax()]

v_precio = prod_max_precio.values
v_ventas = prod_max_ventas.values

# Calculamos sus normas L1 y L2
norma_l1_precio = np.linalg.norm(v_precio, ord=1)
norma_l2_precio = np.linalg.norm(v_precio, ord=2)
norma_l1_ventas = np.linalg.norm(v_ventas, ord=1)
norma_l2_ventas = np.linalg.norm(v_ventas, ord=2)

# 3. INTERPRETACIÓN PEDAGÓGICA
print(f"Producto más caro: {v_precio}, Normas L1={norma_l1_precio:.2f}, L2={norma_l2_precio:.2f}")
print(f"Producto más vendido: {v_ventas}, Normas L1={norma_l1_ventas:.2f}, L2={norma_l2_ventas:.2f}")
print("\nLa norma nos da una medida de la 'magnitud' total de un vector. Un producto con una norma L2 muy alta es un 'extremo' en el espacio de productos, alejado del origen (un producto con precio 0 y ventas 0).")

# 4. VISUALIZACIÓN
plot_vectors_2d(
    vectors=[v_precio, v_ventas],
    legends=['Max Precio', 'Max Ventas'],
    title='Vectores de Productos Extremos'
)

### Ejemplo Demostrativo 3: Operaciones con el "Hilo Conductor"

In [ ]:
# 1. DATOS
# Usamos 'datos_estudiantes'

# 2. APLICACIÓN DEL CONCEPTO
# Identifiquemos un estudiante de bajo y alto rendimiento
est_bajo = datos_estudiantes.loc[datos_estudiantes['calificacion_examen'].idxmin()].values
est_alto = datos_estudiantes.loc[datos_estudiantes['calificacion_examen'].idxmax()].values

# El 'vector de mejora' es la diferencia entre ellos
v_mejora = est_alto - est_bajo

# 3. INTERPRETACIÓN PEDAGÓGICA
print(f"Estudiante bajo rendimiento: {np.round(est_bajo, 2)}")
print(f"Estudiante alto rendimiento: {np.round(est_alto, 2)}")
print(f"Vector de Mejora necesario: {np.round(v_mejora, 2)}")
print("Este vector nos dice cuánto necesitan aumentar las 'horas de estudio' y la 'calificación' para pasar de un perfil a otro.")

# 4. VISUALIZACIÓN
# Visualizamos la operación v_bajo + v_mejora = v_alto
plot_vectors_2d(
    vectors=[est_bajo, v_mejora, est_alto],
    origins=[[0,0], est_bajo, [0,0]],
    legends=['Bajo Rendimiento', 'Vector Mejora', 'Alto Rendimiento'],
    title='Resta de Vectores como la "Distancia" a Mejorar'
)

---
## 3. Combinaciones Lineales y Span

### Combinación Lineal

Una **combinación lineal** de un conjunto de vectores $\{\vec{v}_1, \dots, \vec{v}_p\}$ es cualquier vector $\vec{y}$ que se pueda escribir como una suma ponderada de ellos:
$$ \vec{y} = c_1\vec{v}_1 + c_2\vec{v}_2 + \dots + c_p\vec{v}_p $$
donde $c_1, \dots, c_p$ son escalares (pesos).

Esta es quizás **la idea más importante del álgebra lineal**: nos dice que podemos construir nuevos vectores "mezclando" un conjunto de vectores base.

### Espacio Generado (Span)

El **Espacio Generado (Span)** de un conjunto de vectores $\{\vec{v}_1, ..., \vec{v}_p\}$ es el conjunto de **TODAS** las posibles combinaciones lineales que se pueden formar con ellos. Es el "universo de lo posible" que puedes construir con tus vectores base.

- El Span de un solo vector (no nulo) en $ℝ^2$ o $ℝ^3$ es una **línea**.
- El Span de dos vectores no alineados en $ℝ^2$ es todo el **plano** $ℝ^2$.
- El Span de dos vectores no alineados en $ℝ^3$ es un **plano** que pasa por el origen.

### Ejemplo Demostrativo 4: Visualización Interactiva del Span

Usa los deslizadores para cambiar los componentes de `v1` y `v2`. Observa cómo el `span` (el área gris) cambia. Si los vectores se alinean (uno es múltiplo del otro), el `span` colapsa de un plano a una simple línea. Esto significa que se han vuelto **linealmente dependientes**.

In [ ]:
@interact
def interactive_span(c1=slider(-2, 2, step_size=0.5, default=1), 
                     c2=slider(-2, 2, step_size=0.5, default=1)):
    
    v1 = np.array([2, 1])
    v2 = np.array([-1, 2])
    
    combination = c1*v1 + c2*v2
    
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Dibujar el span (plano)
    ax.fill([-6, 6, 6, -6], [-6, 6, 6, -6], color='gray', alpha=0.1, label='Span(v1, v2)')
    
    # Dibujar vectores base
    ax.quiver(0, 0, v1[0], v1[1], angles='xy', scale_units='xy', scale=1, color='#0072B2', label='v1')
    ax.quiver(0, 0, v2[0], v2[1], angles='xy', scale_units='xy', scale=1, color='#E69F00', label='v2')
    
    # Dibujar combinación lineal
    ax.quiver(0, 0, combination[0], combination[1], angles='xy', scale_units='xy', scale=1, color='#D55E00', 
              label=f'{c1:.1f}*v1 + {c2:.1f}*v2')
    
    ax.set_xlim(-5, 5); ax.set_ylim(-5, 5)
    ax.set_aspect('equal'); ax.grid(True); ax.legend()
    ax.set_title("Explorando Combinaciones Lineales dentro del Span")
    plt.show()

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión paso a paso.

### === EJERCICIO GUIADO 1: Suma de Vectores de Estudiantes ===

In [ ]:
# DATOS: Vectores de los estudiantes en índice 5 y 15.
v_est5 = datos_estudiantes.iloc[5].values
v_est15 = datos_estudiantes.iloc[15].values

# TODO 1: Calcula un vector 'equipo' que sea la suma de los perfiles de ambos estudiantes.
v_equipo = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert v_equipo.shape == (2,), "El vector resultante no tiene la dimensión correcta."
assert np.allclose(v_equipo, v_est5 + v_est15), "El cálculo de la suma no es correcto."
print(f"✅ ¡Correcto! El vector de 'esfuerzo de equipo' es: {np.round(v_equipo, 2)}")

# TODO 2: Visualiza los dos vectores originales y el vector 'equipo'.
# PISTA: Usa plot_vectors_2d con una lista de 3 vectores.
# COMPLETAR

### === EJERCICIO GUIADO 2: Vector de 'Diferencia de Mercado' ===

In [ ]:
# DATOS: Vectores de dos productos de 'datos_negocio'
v_prod_A = datos_negocio.iloc[10].values # Producto A
v_prod_B = datos_negocio.iloc[20].values # Producto B

# TODO 1: Calcula el vector que representa la diferencia de perfil entre el producto A y el B.
v_diferencia = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert np.allclose(v_prod_A, v_prod_B + v_diferencia), "La resta no es correcta. Recuerda: B + (A-B) = A"
print(f"✅ ¡Correcto! Para ir del perfil B al A, necesitas cambiarlo por el vector {np.round(v_diferencia, 2)}.")

# TODO 2: Visualiza el vector B y el vector diferencia (partiendo desde la punta de B).
# PISTA: Usa los parámetros 'vectors' y 'origins' de plot_vectors_2d.
# plot_vectors_2d(vectors=[...], origins=[...])
# COMPLETAR

### === EJERCICIO GUIADO 3: Calcular Norma L1 (Distancia de Taxi) ===

In [ ]:
# DATOS: Un vector de producto.
v_producto = datos_negocio.iloc[30].values

# TODO 1: Calcula la norma L1 del vector v_producto.
# PISTA: Usa np.linalg.norm() con el parámetro 'ord=1'.
norma_l1 = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
expected_l1 = np.sum(np.abs(v_producto))
assert np.isclose(norma_l1, expected_l1), "El cálculo de la norma L1 no es correcto."
print(f"✅ ¡Correcto! La norma L1 (Manhattan) del producto {np.round(v_producto,2)} es {norma_l1:.2f}.")
print("Interpretación: Si un taxi viajara desde el origen (0,0) hasta el punto del producto, recorrería esta distancia total moviéndose por los ejes 'precio' y 'ventas'.")

### === EJERCICIO GUIADO 4: Normalizar un Vector (Crear Vector Unitario) ===

In [ ]:
# DATOS: Un vector de estudiante.
v_estudiante = datos_estudiantes.iloc[42].values

# TODO 1: Calcula la norma L2 (Euclidiana) del vector.
# PISTA: Usa np.linalg.norm() con 'ord=2' o sin el parámetro 'ord'.
norma_l2 = # COMPLETAR

# TODO 2: Crea un 'vector de perfil' (unitario) dividiendo el vector original por su norma L2.
v_unitario = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert norma_l2 > 0, "La norma debe ser positiva para poder dividir."
assert np.isclose(np.linalg.norm(v_unitario), 1.0), "El vector resultante no es unitario. Su norma L2 debería ser 1."
print(f"✅ ¡Correcto! El perfil del estudiante (vector unitario) es {np.round(v_unitario, 2)}.")
print("Interpretación: El vector unitario captura solo la 'dirección' o 'perfil' del estudiante, ignorando la magnitud total de sus componentes.")

# Visualiza el vector original y su versión normalizada. Observa que apuntan en la misma dirección.
plot_vectors_2d([v_estudiante, v_unitario], legends=['Original', 'Unitario (Perfil)'])

### === EJERCICIO GUIADO 5: Construir una Combinación Lineal Específica ===

In [ ]:
# DATOS: Dos vectores base.
v1 = np.array([3, 1])
v2 = np.array([1, 2])

# TODO 1: Crea un nuevo vector 'v_comb' que sea la combinación lineal 1.5*v1 - 2*v2.
v_comb = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert np.allclose(v_comb, np.array([2.5, -2.5]))
print(f"✅ ¡Correcto! El vector resultante es {v_comb}")

# TODO 2: Visualiza los vectores base v1, v2 y el vector resultante v_comb.
# COMPLETAR

### === EJERCICIO GUIADO 6: Comprobar si un Vector está en el Span ===

In [ ]:
# DATOS: Dos vectores base y un vector objetivo.
v1 = np.array([2, 1])
v2 = np.array([1, -1])
target = np.array([7, 2])

# Queremos saber si existen c1, c2 tales que c1*v1 + c2*v2 = target.
# Esto es un sistema de ecuaciones lineales: A * x = b
# donde A es la matriz con v1 y v2 como columnas, x = [c1, c2], y b = target.

# TODO 1: Crea la matriz A.
# PISTA: Usa np.array([v1, v2]).T para poner los vectores como columnas.
A = # COMPLETAR

try:
    # TODO 2: Resuelve el sistema para encontrar los coeficientes [c1, c2].
    # PISTA: Usa np.linalg.solve(A, target).
    coefs = # COMPLETAR
    print(f"✅ ¡Sí! El vector está en el span. Coeficientes: c1={coefs[0]:.2f}, c2={coefs[1]:.2f}")
except np.linalg.LinAlgError:
    print("❌ No, el vector no está en el span (o los vectores base son linealmente dependientes).")

# VERIFICACIÓN VISUAL
plot_vectors_2d([v1, v2, target], legends=['v1', 'v2', 'target'])

### === EJERCICIO GUIADO 7: El Span de Vectores Colineales ===

In [ ]:
# DATOS: Dos vectores que están en la misma línea (colineales).
v1 = np.array([2, 1])
v2 = np.array([-4, -2]) # v2 = -2 * v1
target_off_line = np.array([3, 3]) # Un vector que NO está en la línea

# TODO 1: Intenta resolver el sistema para ver si target_off_line está en el span de v1 y v2.
A = np.array([v1, v2]).T

try:
    # Esta línea debería fallar y lanzar una excepción
    coefs = np.linalg.solve(A, target_off_line)
    print(f"❌ Algo salió mal, se encontró una solución: {coefs}")
except np.linalg.LinAlgError:
    # TODO 2: Imprime un mensaje explicando por qué falló.
    # PISTA: ¿Qué tipo de espacio generan dos vectores colineales?
    print("✅ ¡Correcto! El sistema no tiene solución.")
    print("Interpretación: # COMPLETAR CON TU EXPLICACIÓN")


### === EJERCICIO GUIADO 8: Error Robusto (L1) vs. Sensible a Outliers (L2) ===

In [ ]:
# DATOS: Dos vectores de error. e2 tiene un gran outlier.
y_true = np.array([2, 3, 4, 5, 6])
preds1 = np.array([2.2, 3.3, 3.8, 5.2, 6.3]) # Errores pequeños y consistentes
preds2 = np.array([2.1, 2.9, 4.1, 5.1, 10]) # Un gran error al final

e1 = y_true - preds1
e2 = y_true - preds2

# TODO 1: Calcula el MAE (basado en norma L1) para ambos errores.
# MAE = ||error||_1 / n
mae1 = # COMPLETAR
mae2 = # COMPLETAR

# TODO 2: Calcula el RMSE (basado en norma L2) para ambos errores.
# RMSE = ||error||_2 / sqrt(n)
rmse1 = # COMPLETAR
rmse2 = # COMPLETAR

print(f"Modelo 1: MAE={mae1:.2f}, RMSE={rmse1:.2f}")
print(f"Modelo 2: MAE={mae2:.2f}, RMSE={rmse2:.2f}")
print("\nObserva: El MAE de M2 no es mucho peor que el de M1, pero su RMSE es mucho más alto. \nEsto demuestra que RMSE (norma L2) es mucho más sensible a los outliers.")
assert mae2 < rmse2, "El RMSE debería ser mayor que el MAE, especialmente con outliers."

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Operaciones Vectoriales

**A1 (🟢 Fácil):** Dados $\vec{a}=[2,1]$ y $\vec{b}=[1,3]$, calcula y visualiza $\vec{a}+\vec{b}$.

**A2 (🟢 Fácil):** Con los mismos vectores, calcula y visualiza $3\vec{a}$.

**A3 (🟢 Fácil):** Dados $\vec{u}=[1,2,3]$ y $\vec{v}=[-2,1,0]$, calcula $2\vec{u} + 3\vec{v}$.

**A4 (🟡 Medio):** Si $\vec{a} + \vec{b} = [5, 6]$ y $\vec{a} = [1, 4]$, calcula el vector $\vec{b}$.

**A5 (🟡 Medio):** Usando `datos_estudiantes`, toma los vectores de los primeros dos estudiantes. Calcula un vector "estudiante promedio" que sea $0.5 * est_1 + 0.5 * est_2$. Visualiza los tres.

**A6 (🟡 Medio):** Usando `datos_negocio`, toma los vectores de dos productos (índices 5 y 6). Imagina que se fusionan en un nuevo producto cuyo perfil es la suma de los dos. Calcula y visualiza este nuevo vector de 'producto fusionado'.

**A7 (🔴 Reto):** El perfil de un estudiante es `v = [10, 80]`. Un programa de tutoría promete una mejora representada por el vector `m = [5, 7]`. Un programa de deporte reduce las horas de estudio pero aumenta el bienestar (reflejado en la nota) con un vector `d = [-2, 3]`. Si el estudiante hace ambos, ¿cuál es su perfil final? Visualiza el camino: `v -> v+m -> v+m+d`.

### Parte B: Normas Vectoriales

**B1 (🟢 Fácil):** Calcula la norma L1 y L2 del vector $\vec{v}=[3,4]$.

**B2 (🟢 Fácil):** Calcula la norma L1 y L2 del vector de estudiante en el índice 20 de `datos_estudiantes`.

**B3 (🟡 Medio):** ¿Para qué vector es mayor la diferencia entre su norma L1 y L2: para $\vec{a}=[10, 1]$ o para $\vec{b}=[7, 7]$? ¿Por qué crees que ocurre esto?

**B4 (🟡 Medio):** Encuentra el producto en `datos_negocio` con la mayor norma L2. ¿Qué características tiene este producto (precio alto, ventas altas, o ambos)?

**B5 (🟡 Medio):** Normaliza el vector del producto con la mayor norma L2 que encontraste en el ejercicio anterior. Visualiza el vector original y el normalizado.

**B6 (🔴 Reto):** Genera datos con outliers usando `create_edge_cases(rng, case_type='outliers')`. Calcula el vector promedio de todos los puntos. Ahora, calcula la distancia L1 y L2 de cada punto a este promedio. ¿Qué punto tiene la mayor distancia L2? ¿Es el mismo que tiene la mayor distancia L1?

**B7 (🔴 Reto Conceptual):** Imagina que estás en $ℝ^3$. La norma L2 representa la distancia directa. Describe un camino entre el origen (0,0,0) y el punto (3,4,5) cuya longitud sea igual a la norma L1 de ese vector.

### Parte C: Combinaciones Lineales y Span

**C1 (🟢 Fácil):** Expresa el vector $\vec{w}=[3,5]$ como una combinación lineal de los vectores base canónicos $\vec{i}=[1,0]$ y $\vec{j}=[0,1]$.

**C2 (🟢 Fácil):** Dados $\vec{a}=[2,1]$ y $\vec{b}=[0,3]$, calcula y visualiza la combinación lineal $2\vec{a} + 1\vec{b}$.

**C3 (🟡 Medio):** ¿El vector $\vec{c}=[4, 5]$ está en el span de $\vec{a}=[2, 1]$ y $\vec{b}=[0, 3]$? Si es así, encuentra los coeficientes.

**C4 (🟡 Medio):** Describe geométricamente el span de los vectores $\vec{u}=[1, 2, 3]$ y $\vec{v}=[2, 4, 6]$ en $ℝ^3$. ¿Generan una línea o un plano? ¿Por qué?

**C5 (🟡 Medio):** Genera 50 puntos de una 'línea' usando `create_geometric_shapes`. Toma los dos primeros puntos como vectores $\vec{v1}$ y $\vec{v2}$. ¿Son linealmente independientes? ¿Cuál es su span? Ahora toma el último punto, $\vec{v3}$. ¿Está en el span de los dos primeros?

**C6 (🔴 Reto):** Encuentra los escalares $c_1$ y $c_2$ tales que $c_1[1, 2] + c_2[3, 1] = [-1, 1]$.

**C7 (🔴 Reto):** Tienes dos perfiles base de producto: 'Barato y Popular' $\vec{b}=[20, 800]$ y 'Caro y de Nicho' $\vec{n}=[400, 50]$. ¿Puedes aproximar el perfil del producto en el índice 40 de `datos_negocio` como una combinación lineal de $\vec{b}$ y $\vec{n}$? Encuentra los coeficientes aproximados.

**C8 (🔴 Reto Conceptual):** ¿El span de tres vectores en $ℝ^3$ es siempre todo $ℝ^3$? Diseña un ejemplo numérico donde no lo sea y explica por qué.

---

## ✅ Mini-Quiz de Autoevaluación

1. ¿Cuál es la diferencia de sensibilidad a outliers entre la norma L1 y L2?
2. ¿Qué significa geométricamente que un vector $\vec{w}$ esté en el Span de los vectores $\vec{v_1}$ y $\vec{v_2}$ en $ℝ^3$ (asumiendo que $\vec{v_1}$ y $\vec{v_2}$ no son colineales)?
3. Verdadero o Falso: La suma vectorizada en NumPy es más rápida que un bucle `for` en Python principalmente porque las CPUs modernas tienen instrucciones especiales para operaciones matemáticas en paralelo sobre arrays de datos.
4. Si normalizas un vector (dividiéndolo por su norma L2), ¿qué propiedad tendrá siempre el vector resultante?

## 🚀 Próximos Pasos

¡Excelente trabajo! Has dominado las operaciones fundamentales, has entendido su impacto en el rendimiento computacional y ahora puedes ver cómo los conceptos de Combinación Lineal y Span forman el verdadero lenguaje del Álgebra Lineal.

- En el siguiente notebook, **`1.1.2.1_Matrices_y_Vectores`**, pasaremos de tratar con vectores individuales a organizarlos en una estructura superior: la **matriz**. Verás que una matriz es simplemente una colección de vectores (fila o columna) y que conceptos como el Span se vuelven aún más poderosos y centrales para la ciencia de datos.